In [28]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm_notebook
import pandas as pd

In [7]:
feature_matrix = np.load('new_feature_matrix.npy')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
cuda = True if torch.cuda.is_available() else False
drop_rate = 0.5
time_step = 5
epochs = 200
feature_matrix = np.load('new_feature_matrix.npy')

In [12]:
scaler = StandardScaler()
## normalization
def normalization(feature_matrix):
    feature_mat = np.zeros((feature_matrix.shape[0],feature_matrix.shape[1],feature_matrix.shape[2]))
    target = np.zeros((feature_matrix.shape[0],feature_matrix.shape[1]))
    for i in range(feature_matrix.shape[0]):
        feature_vector=feature_matrix[i]
        f1 = scaler.fit_transform(feature_vector[:,0].reshape(-1,1)).reshape(-1,1)
        f2 = scaler.fit_transform(feature_vector[:,1].reshape(-1,1)).reshape(-1,1)
        f3 = scaler.fit_transform(feature_vector[:,2].reshape(-1,1)).reshape(-1,1)
        f4 = scaler.fit_transform(feature_vector[:,3].reshape(-1,1)).reshape(-1,1)
        visit_nums = scaler.fit_transform(feature_vector[:,4].reshape(-1,1)).reshape(-1,1)
        feature_vector = np.stack([f1,f2,f3,f4,visit_nums],axis=1).reshape(feature_matrix.shape[1],feature_matrix.shape[2])
        feature_mat[i] = feature_vector
    return feature_mat


## split into train, test and validation
features = feature_matrix
features = normalization(features)
train_idx, test_idx, val_idx = [],[],[]
for i in range(features.shape[0]):
    if i%5<=2:
        train_idx.append(i)
    elif i%5==3:
        val_idx.append(i)
    else:
        test_idx.append(i)

train_features = features[train_idx,:,:]
test_features = features[test_idx,:,:]
val_features = features[val_idx,:,:]

In [15]:
print(val_features.shape)
print(test_features.shape)
print(train_features.shape)

(18, 172, 5)
(18, 172, 5)
(54, 172, 5)


In [14]:
def get_features(features, index):
    f = features[:,index,:].cpu()
    new = np.zeros((len(f) - time_step, time_step, 4-1))
    target = np.zeros((len(f) - time_step, 1))
    for i in range(len(f) - time_step):
        new[i] = f[i:i+time_step,1:4]
        target[i] = f[i+time_step, 4]
    return new, target

# LSTM(Long Short Term Memory)

In [21]:
class MLP(torch.nn.Module):
    def __init__(self, n_i, n_h, n_o):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(n_i, n_h)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(n_h, n_o)
    def forward(self, input):
        input = self.linear1(input)
        input = self.relu(input)
        input = F.dropout(input, p=drop_rate, training=self.training)
        input = self.linear2(input)
        return input
###Convert the data to tensor 
def totensor(data):
    return torch.Tensor(data).to(device)

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        self.mlp = MLP(time_step*4,32,1)#input is the num of time_step
        self.lstm1 = nn.LSTM(input_size=3,hidden_size=128,num_layers=2,batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128,hidden_size=4,num_layers=2,batch_first=True)
    def forward(self, x):
        x1,x2,x3 = x.size()
        h1, (h1_T,c1_T) = self.lstm1(x)
        h2, (h2_T, c2_T) = self.lstm2(h1)
        h2 = h2.reshape(x1,-1)
        output= self.mlp(h2)
        return output 

In [24]:
min_val_loss = np.inf
mean_losses = []
losses = pd.DataFrame(columns = ['Training Loss', 'Validating Loss','Testing Loss'])
model = LSTM().cpu()
loss = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(),lr=0.001)

In [29]:
for epoch in tqdm_notebook(range(1, epochs + 1)):
    mean_loss, n = 0.0 , train_features.shape[1]
    train_mean_loss, test_mean_loss = 0, 0
    for i in range(n):
      # Tensor.cpu()
        train, train_target = get_features(totensor(train_features), i)
        train, train_target = totensor(train), totensor(train_target)
        val, val_target = get_features(totensor(val_features), i)
        val, val_target = totensor(val), totensor(val_target)
        y_train, y_val = model(train), model(val)
        train_loss, val_loss = loss(y_train, train_target), loss(y_val, val_target)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        losses.loc[i,'Training Loss'], losses.loc[i,'Validating Loss'] = train_loss.detach().cpu().numpy(), val_loss.detach().cpu().numpy()
    train_loss, val_loss = losses['Training Loss'].mean(), losses['Validating Loss'].mean()
    mean_losses.append((train_loss, val_loss))
    if train_loss < min_val_loss:
        min_val_loss = train_loss
    print('Epoch: {:03d} | Lr: {:.20f} |Train loss: {:.8f}|Val loss: {:.8f}'.\
          format(epoch, optimizer.param_groups[0]['lr'], min_val_loss, val_loss))

print('\nTraining finished.\n')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 001 | Lr: 0.00100000000000000002 |Train loss: 1.00992788|Val loss: 1.01476971
Epoch: 002 | Lr: 0.00100000000000000002 |Train loss: 1.00992788|Val loss: 1.04643409
Epoch: 003 | Lr: 0.00100000000000000002 |Train loss: 1.00992788|Val loss: 1.03875395
Epoch: 004 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.00986144
Epoch: 005 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.07259697
Epoch: 006 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.04745652
Epoch: 007 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.03473672
Epoch: 008 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.02873363
Epoch: 009 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.02114371
Epoch: 010 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.02224891
Epoch: 011 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|Val loss: 1.01854520
Epoch: 012 | Lr: 0.00100000000000000002 |Train loss: 1.00985567|V

Epoch: 098 | Lr: 0.00100000000000000002 |Train loss: 0.68074998|Val loss: 0.75987217
Epoch: 099 | Lr: 0.00100000000000000002 |Train loss: 0.68074998|Val loss: 0.70437937
Epoch: 100 | Lr: 0.00100000000000000002 |Train loss: 0.67999685|Val loss: 0.67151921
Epoch: 101 | Lr: 0.00100000000000000002 |Train loss: 0.67999685|Val loss: 0.69723768
Epoch: 102 | Lr: 0.00100000000000000002 |Train loss: 0.66885669|Val loss: 0.67123852
Epoch: 103 | Lr: 0.00100000000000000002 |Train loss: 0.66885669|Val loss: 0.72929063
Epoch: 104 | Lr: 0.00100000000000000002 |Train loss: 0.66885669|Val loss: 0.67740875
Epoch: 105 | Lr: 0.00100000000000000002 |Train loss: 0.66808195|Val loss: 0.66846351
Epoch: 106 | Lr: 0.00100000000000000002 |Train loss: 0.66808195|Val loss: 0.68190122
Epoch: 107 | Lr: 0.00100000000000000002 |Train loss: 0.66808195|Val loss: 0.70889260
Epoch: 108 | Lr: 0.00100000000000000002 |Train loss: 0.66808195|Val loss: 0.77175566
Epoch: 109 | Lr: 0.00100000000000000002 |Train loss: 0.66808195|V

Epoch: 195 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.62597723
Epoch: 196 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.63560965
Epoch: 197 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.62675392
Epoch: 198 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.60685668
Epoch: 199 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.65037736
Epoch: 200 | Lr: 0.00100000000000000002 |Train loss: 0.58925127|Val loss: 0.59036029

Training finished.

